<img src="./images/output-0.png" />

<img src="./images/output-1.png" />

<img src="./images/output-2.png" />

<img src="./images/output-4.png" />

<img src="./images/output-5.png" />

<img src="./images/output-6.png" />

<img src="./images/output-7.png" />

<img src="./images/output-8.png" />

<img src="./images/output-9.png" />

<img src="./images/output-10.png" />

<img src="./images/output-11.png" />

<img src="./images/output-12.png" />

<img src="./images/output-13.png" />

<h1 align=center>basic_webcam.py</h1>
```python
# Example One, getting frames off a webcam.
# basic_webcam.py
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    cv2.imshow('Basic Web Cam',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
```

<h1 align=center>advance_webcam.py</h1>
```python
# Example two, advance camera properties. 
import numpy as np
import cv2
import time

pass 
# create a map to keep track of all these names
prop_map = {
    "pos_msec":cv2.cv.CV_CAP_PROP_POS_MSEC,
    "pos_frame":cv2.cv.CV_CAP_PROP_POS_FRAMES,
    "avi_ratio":cv2.cv.CV_CAP_PROP_POS_AVI_RATIO ,
    "width":cv2.cv.CV_CAP_PROP_FRAME_WIDTH ,
    "height":cv2.cv.CV_CAP_PROP_FRAME_HEIGHT ,
    "fps":cv2.cv.CV_CAP_PROP_FPS ,
    "fourcc":cv2.cv.CV_CAP_PROP_FOURCC ,
    "frame_count":cv2.cv.CV_CAP_PROP_FRAME_COUNT,
    "format":cv2.cv.CV_CAP_PROP_FORMAT ,
    "mode":cv2.cv.CV_CAP_PROP_MODE ,
    "brightness":cv2.cv.CV_CAP_PROP_BRIGHTNESS ,
    "contrast":cv2.cv.CV_CAP_PROP_CONTRAST ,
    "saturation":cv2.cv.CV_CAP_PROP_SATURATION,
    "hue":cv2.cv.CV_CAP_PROP_HUE ,
    "gain":cv2.cv.CV_CAP_PROP_GAIN ,
    "exposure":cv2.cv.CV_CAP_PROP_EXPOSURE ,
    "convert_rgb":cv2.cv.CV_CAP_PROP_CONVERT_RGB ,
 #   "white_balance":cv2.cv.CV_CAP_PROP_WHITE_BALANCE ,
    "rectification":cv2.cv.CV_CAP_PROP_RECTIFICATION}

# get a camera property
def get_prop(cam,name,prop_map):
    return cam.get(prop_map[name])

# set a camera property
def set_prop(cam,name,prop_map,value):
    cam.set(prop_map[name],value)

# print out all of the properites
def poll_props(cam,prop_map):
    out_map = {}
    for k,v in prop_map.items():
        result = cam.get(v)
        if( result == -1.0 ):
            out_map[k] = None
        else:
            out_map[k] = result
    return out_map

# create a camera and get its property
cam = cv2.VideoCapture(0)
properties = poll_props(cam,prop_map)

# list our properties
for k,v in properties.items():
    print "{0:<12}\t:{1:<12}".format(k,v)

while(True):
    # toggle properties and get results. 
    sat = get_prop(cam,"saturation",prop_map)
    if( sat > 0.5 ):
        set_prop(cam,"saturation",prop_map,0.1)
    else:
        set_prop(cam,"saturation",prop_map,1.0)
    time.sleep(0.05)
    ret, frame = cam.read()
    cv2.imshow('Basic Web Cam',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# put our toys back on the shelf 
cam.release()
cv2.destroyAllWindows()
```

<h1 align=center> Go Pro Cameras</h1>
<img src="/files/images/gopro.jpg" align=center>
<h2>
<ul align=left>
<li>All GoPro Cameras through 3+ use a similar interface.</li>
<br>
<li>Each camera acts as a wifi access point with a simple web server.</li>
<br>
<li>We can hack this interface for nefarious purposes.</li>
<br>
<li>Default path is http://10.5.5.9:8080/</li>
<br>
<li><a href="http://goprohero.readthedocs.org/en/latest/">GoProHero</a> is a nice library wrapping a lot of commands.</li>
</h2>
</ul>



<h1 align=center>Some Specifics on GoPros</h1>
<br>
<img src="/files/images/gopro_screen.png" align=center>
<h2>
<ul align=left>
<li>Interface is hitting specific routes with parameters.</li>
<br>
<li>Images and video are collected using urllib and beautiful soup.</li>
<br>
<li>Naming conventions are a little convoluted -- need some helper code.</li>
<br>
<li>Control over resolutions and frame rates. Gain and exposure not so much.</li>
<br>
<li>Live streaming is a bit wonky.</li>
</h2>
</ul>

In [1]:
from goprohero import GoProHero # helper lib
import urllib # used to "wget" files
import bs4 # beautiful soup, to parse file
import time
from IPython.display import Image

def download_and_save(name,route="http://10.5.5.9:8080/videos/DCIM/100GOPRO/"):
    """
    Download a specific file on a connected GoPro camera.
    """
    grab = route+name
    result = urllib.urlopen(grab) # basically wget
    if( result.code == 200 ): # 200 means OK
        with open(name,'wb') as fp: 
            fp.write(result.read()) # write to file
    result.close()

def get_new(last=[],url="http://10.5.5.9:8080/videos/DCIM/100GOPRO/"):
    """
    Find files recently added to our go pro. Requires a list of files
    """
    unique = None
    last = set(last)
    out = urllib.urlopen(url)
    if(out.code == 200):
        soup = bs4.BeautifulSoup(out.read()) # read the file tree
        # make a set of all <a href> in the tree
        fresh = set([row.renderContents() for row in soup.findAll('a')]) 
        # do a set difference with our initial set
        unique = list(fresh.difference(last))
    return unique



In [5]:
# check the camera file system
out = get_new()
# setup our controller
cam = GoProHero(password="herpderp")
# put the camera in burst mode
cam.command('mode','burst')
# tell it to take a shoth
cam.command('record','on')
# wait to finish
time.sleep(3)
# get the new files
out = get_new(last=out)
for o in out:
    print o
    download_and_save(o)

KeyboardInterrupt: 

In [ ]:
Image(out[0])


<h1 align=center>A lot of cameras work just like our GoPro</h1>
<img src="/files/images/phone_interface.png" align=center>
<h2>
<ul align=left>
<li>Most newer CCTV cameras, baby monitors, etc. are IP web cameras.</li>
<br>
<li>Each camera acts as a little web server and streams images.</li>
<br>
<li>Even better, every one of us has one in our pocket!</li>
<br>
<li>A large variety of Android/IOS apps as cameras.</li>
<br>
<li>Harder to write control interface, but generally easy to stream.</li>
</h2>
</ul>

<h1 align=center>A lot of cameras work just like our GoPro</h1>
<img src="/files/images/web_interface.png" align=center>
<h2>
<ul align=left>
<li>Most of these apps support <a href="">motion jpeg</a>.</li>
<br>
<li>MJPEG simply dumps a series of jpegs, i.e. only *intra* frame compression.</li>
<br>
<li>The trick is to find the begining and end of each file.</li>
<br>
<li>Ususally of very low quality to support streaming.</li>
<br>
<li>There are <a href="http://www.insecam.org/en/view/279373/">lots of them on the web.</a></li>
</h2>
</ul>

<h1 align=center>streaming_example.py</h1>
```python
"""
Read and display a simple MJPEG stream given a URL
""" 
import cv2
import urllib 
import numpy as np
# open our url stream
stream=urllib.urlopen('http://192.168.1.228:8080/video')
# create a buffer of bytes
bytes=''
invert = False
while True:
    # read some bytes from the stream
    # how much to read is a matter of taste
    bytes+=stream.read(1024*8)
    # \xff\xd8 is the start of a jpg
    # \xff\xd9 is the end of a jpg
    a = bytes.find('\xff\xd8')
    b = bytes.find('\xff\xd9')
    # if we find them
    if a!=-1 and b!=-1:
        jpg = bytes[a:b+2] # create a buffer
        bytes= bytes[b+2:] # save the remaining bytes
        #numpy can handle strings as image data
        img = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),cv2.CV_LOAD_IMAGE_COLOR)
        # Add some bling to show we can process
        if invert:
            img = 255-img
        # show our image
        cv2.imshow("live_and_direct",img)
    # MVP keyboard input
    my_key = cv2.waitKey(1)
    if my_key & 0xFF == ord('q'):
        break
    if my_key & 0xFF == ord('i'):
        invert = not invert 
# put our toys away when done. 
stream.close()
cv2.destroyAllWindows()
```

<h1 align=center> Cameras, Cameras, Everywhere</h1>
<img src="/files/images/youtube_live.png" align=center>
<h2>
<ul align=left>
<li>Lots of cool streaming sources, Youtube, Twtich, Ustream, etc.</li>
<br>
<li>Can we treat them like a camera or use our own?</li>
<br>
<li>Parsing the urls is kinda tough, python can help.</li>
<br>
<li><a href="http://docs.livestreamer.io/install.html">Livestreamer</a> is a tool for parsing stream urls</li>
<br>
<li>Has some painful dependencies, but installed relatively easy.</li>
</h2>
</ul>

<h1 align=center> Streaming vs Progressive Downloading</h1>
<img src="/files/images/youtube_live.png" align=center>
<h2>
<ul align=left>
<li>Live streamer can parse and point us to a file buffer.</li>
<br>
<li>What's in this buffer?</li>
<br>
<li>Turns out is mainly a series of small mpeg files. Streaming isn't streaming, it is progressive downloading.</li>
<br>
<li>You get the head of a file and append clips at the end.</li>
<br>
<li>It acutally plays well with OpenCV!</li>
<br>
<li>This example is simple, in real applications we should use a threading model.</li>
</h2>
</ul>

<h1 align=center>live_stream_service.py</h1>
```python
import cv2
import numpy as np
import time
import livestreamer

# use live streamer to figure out the stream info
streams = livestreamer.streams("http://www.youtube.com/watch?v=-JcaDowz2G8&ab_channel=HopeCollegeAthletics")
stream = streams['best']
# open our out file. 
fname = "test.mpg"
vid_file = open(fname,"wb")
# dump from the stream into an mpg file -- get a buffer going
fd = stream.open()
for i in range(0,1048):
    if i%256==0:
        print "Buffering..."
    new_bytes = fd.read(1024)
    vid_file.write(new_bytes)
# open the video file from the begining
print "Done buffering."
cam = cv2.VideoCapture(fname)
while True:
    ret, img = cam.read()                      
    try:
        if ret:
            img = 255-img # invert the colors
            cv2.imshow('live_img',img)
    except:
        print "DERP"
        continue
    if (0xFF & cv2.waitKey(5) == 27) or img.size == 0:
        break
    time.sleep(0.05)
    # dump some more data to the stream so we don't run out. 
    new_bytes = fd.read(1024*8)
    vid_file.write(new_bytes)
vid_file.close()
fd.close()
del fd
del vid_file
del new_bytes
```

<h1 align=center>Digital Single-Lens Reflex (DSLR) Cameras</h1>
<img src="/files/images/DSLR.jpg" align=center>
<h2>
<ul align=left>
<li>Fairly common camera type with large sensors.</li>
<br>
<li>Lots and lots of different lenses and accessories.</li>
<br>
<li>Allow precise control of optics and camera parameters</li>
<br>
<li>Bad: images are large and interface is vendor specific.</li>
<br>
<li>Good: raw data and resolution give us lots of data to play with.</li>
</h2>
</ul>

<h1 align=center>Digital Single-Lens Reflex (DSLR) Cameras</h1>
<img src="/files/images/animated.gif" align=center>
<h2>
<ul align=left>
<li><a href="http://www.gphoto.org/proj/gphoto2/">GPhoto2</a> library is good for Canon brand cameras.</li>
<br>
<li>Interface with swig, heavy handed but it works.</li>
<br>
<li>Setting must match camera modes.</li>
<br>
<li>Use cases: HDR, Computational Photography, Science!</li>
</h2>
</ul>

<h1 align=center>DLSR_Example.py</h1>
```python
import logging
import os
import subprocess
import sys
import gphoto2 as gp
import time

def setup():
    """
    Attempt to attach to a gphoto device and grab the camera and context. Return the results.
    """
    logging.basicConfig(
        format='%(levelname)s: %(name)s: %(message)s', level=logging.WARNING)
    gp.check_result(gp.use_python_logging())
    context = gp.gp_context_new()
    camera = gp.check_result(gp.gp_camera_new())
    gp.check_result(gp.gp_camera_init(camera, context))
    text = gp.check_result(gp.gp_camera_get_summary(camera, context))
    print text.text
    return camera,context


def recurse_config(child,params={}):
    """
    The gphoto control structure is a byzantine swig structure.
    This function traverses it recursively and puts it in a
    nice python dictionary.
    """
    if(child.count_children() <= 0):
        my_choices = []
        try:
            n = child.count_choices()
            if( n > 0 ):
                for k in range(0,n):
                    my_choices.append(child.get_choice(int(k)))
        except:
            return my_choices
        return my_choices
    else:
        for i in range(0, child.count_children()):
            chill = child.get_child(i)
            name = chill.get_name()
            params[name] = recurse_config(chill,{})
        return params

def print_config_dict(cdict,lvl=""):
    """
    Print a the python config dictionary for a camera.
    """
    if isinstance(cdict,dict):
        for k,v in cdict.items():
            print "{0}{1}".format(lvl,k)
            print_config_dict(v,lvl+"\t")
    elif isinstance(cdict,list):
        for l in cdict:
            print "{0}{1}".format(lvl,l)
        return

def set_config(camera,context,config,path,value):
    """
    Given a gphoto camera, context, and config 
    traverse the path of the config tree and set
    a parameter value. The path is basically the nodes
    to address in the control structure. Once the config
    object has been modified we have to set it on the camera.
    """
    current = config
    for p in path:
        current = current.get_child_by_name(p)
    current.acquire()
    current.set_value(value)
    current.disown()
    gp.check_result(gp.gp_camera_set_config(camera,config, context))
    print "Set {0} to {1}".format(current.get_name(),current.get_value())


def capture_image(camera,context,name):
    """
    Use gphoto to capture an image and retrieve it.
    Place the file in /tmp/name
    """
    file_path = gp.check_result(gp.gp_camera_capture(
        camera, gp.GP_CAPTURE_IMAGE, context))
    target = os.path.join('/tmp', name)
    print 'Copying image to {0}'.format(target)
    camera_file = gp.check_result(gp.gp_camera_file_get(
            camera, file_path.folder, file_path.name,
            gp.GP_FILE_TYPE_NORMAL, context))
    gp.check_result(gp.gp_file_save(camera_file, target))
    gp.check_result(gp.gp_camera_exit(camera, context))

        
def main():
    # set up our camera.
    camera,context = setup()
    # grab a single test image. 
    capture_image(camera,context,"crap.jpg")
    # Get the configuration of the camera
    config = gp.check_result(gp.gp_camera_get_config(camera, context))
    # Pythonify and print the configuration of the camera so
    # we can see what parameters we can play with. 
    pconfig = recurse_config(config)
    print_config_dict(pconfig)
    # Put the camera in AV mode, or aperture priority. 
    # Camera needs this to fiddle with aperture. 
    set_config(camera,context,config,["capturesettings","autoexposuremode"],"AV")
    count = 0
    # for all of the available aperture settings...
    for param in pconfig["capturesettings"]["aperture"]:
        # get the camera configuration
        config = gp.check_result(gp.gp_camera_get_config(camera, context))
        # set the new configuration
        set_config(camera,context,config,["capturesettings","aperture"],param)
        # and capture an image.
        fname = "Capture{:0>5}.jpg".format(count)
        capture_image(camera,context,fname)
        count += 1

if __name__ == "__main__":
    sys.exit(main())
```

<h1 align=center>Machine Vision Cameras and Robot Operating System</h1>
    <img src="/files/images/ROS.png" align=center>
<h2>
<ul align=left>
<li>There are lots of industrial machine vision cameras out there.</li>
<br>
<li>Vendors like Allied, Basler, IDS, Point Grey, Matrox, Teledyne Dalsa, Axis, FLIR.</li>
<br>
<li>Each camera can address specific application needs.</li>
<br>
<li>There cameras can have esoteric interfaces (GigE, Firewire, etc).</li>
<br>
<li>How to get the into Python?</li>
</h2>
</ul>

<h1 align=center>Why ROS?</h1>
<h2>
<ul align=left>
<li>Roboticists use these cameras, so there are a lot of packages.</li>
<br>
<li>The ROS allows you to use C++ packages from python easily.</li>
<br>
<li>Lots of tools for viewing, logging, controlling, debugging, and calibrating cameras.</li>
<br>
<li>Package management for camera libraries and native OpenCV support.</li>
<br>
<li>Con: a lot of stuff to learn and install, but worth it (see previous talks).</li>
<br>
<li>ROS uses a message bus, python can poll and controll cameras from this bus.</li>
</h2>
</ul>

<h1 align=center>What we are going to do.</h1>
<img src="/files/images/threshold.png" align=center>   
<h2>
<ul align=left>
<li>Get depth images from a structure sensor (depth camera).</li>
<br>
<li>Use the OpenNI C++ package to interface to the structure sensor.</li>
<br>
<li>ROS publishes images to a ROS topic.</li>
<br>
<li>We are going to subscribe to this bus and process the images.</li>
<br>
<li>Take a look at some of the tools.</li>
</h2>
</ul>

<h1 align=center>A ROS Node Example</h1>
```python
#!/usr/bin/env python
import rospy
# This is the tool that marshals images into OpenCV
from cv_bridge import CvBridge, CvBridgeError 
# Import some stock ROS message types.
from sensor_msgs.msg import Image
# import some utils.
import numpy as np
import cv
import SimpleCV as scv
import copy as copy

class ProcessDepth:
    def __init__(self):
        # Allows conversion between numpy arrays and ROS sensor_msgs/Image
        self.bridge = CvBridge() 

        # Allow our topics to be dynamic.
        self.input_camera_topic = rospy.get_param('~input_camera_topic', '/camera/depth/image_rect')
        self.output_camera_topic = rospy.get_param('~output_camera_topic', '/processed')

        # WE are going to publish a debug image as it comes in.
        self.pub = rospy.Publisher(self.output_camera_topic, Image,queue_size=10)
        rospy.Subscriber(self.input_camera_topic, Image, self._process_depth_img)
        # run the node
        self._run()

    # Keep the node alive
    def _run(self):
        rospy.spin()
            
    def _process_depth_img(self,input):
        # convert our image to CV2 numpy format from ROS format
        latest_image = self.bridge.imgmsg_to_cv2(input)

        if( latest_image is not None ):
            try:
                # convert the image to SimpleCV
                # The input image is single channel float and we want rgb uint8
                # it is also full of nasty nans. We get the min and max and scale
                # the image from [0,flt_max] to [0,255]
                dmin = np.nanmin(latest_image)
                dmax = np.nanmax(latest_image)
                latest_image = latest_image - dmin
                sf = 255.0/(dmax-dmin)
                latest_image = sf*latest_image
                # Convert to uint8
                temp = latest_image.astype(np.uint8)
                # move to SimpleCV RGB
                img = scv.Image(temp, cv2image=True, colorSpace=scv.ColorSpace.RGB)
                # get values less than 128
                lt = img.threshold(128).invert()
                # get values greater than 64
                gt = img.threshold(64) 
                # do the logical and of the two depths
                range = lt*gt
                # apply the mask to the input image
                blobs = img.findBlobsFromMask(mask=range)
                # draw the results. 
                if( blobs ):
                    blobs.draw(color=scv.Color.RED,width=-1)
                img = img.applyLayers()
                # convert SimpleCV to CV2 Numpy Format
                cv2img = img.getNumpyCv2()
                # Convert Cv2 numpy to ROS format
                img_msg = self.bridge.cv2_to_imgmsg(cv2img, "bgr8")
                # publish the topic.
                self.pub.publish(self.bridge.cv2_to_imgmsg(cv2img, "bgr8"))
            except CvBridgeError, e:
                rospy.logwarn("PROCESSING EXCEPTION {0}".format(e))

# Boilerplate node spin up. 
if __name__ == '__main__':
    try:
        rospy.init_node('ProcessDepth')
        p = ProcessDepth()
    except rospy.ROSInterruptException:
        pass

```